# 麥克連大盤脈動_part_3_美股大盤收盤資訊  
* [StockQ.org](https://www.stockq.org/stock/history/)  
* [道瓊](https://www.stockq.org/index/INDU.php)
* [群益證券～全球主要指數收盤行情](https://stock.capital.com.tw/z/zn/zna/zna.djhtm)  
* [道瓊指數 from histock](https://histock.tw/index/DJI)  
* [NASDAQ指數 from histock](https://histock.tw/index/NASDAQ)  
* [費城半導體 from histock](https://histock.tw/index/SOX)  
* [台積電ＡＤＲ from histock](https://histock.tw/index/TSM)  
* [美元指數 from histock](https://histock.tw/index/USDX)  

In [1]:
import os
import sys
from pathlib import Path
from io import StringIO
import pandas as pd
import requests
from bs4 import BeautifulSoup
import duckdb

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

## 公用參數設定

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [5]:
# TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
# twstock_db_path = f"{os.getcwd()}\\data\\twstock.duckdb"

In [6]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [ ]:
# ipython 載入 sql 擴充, 並設置 duckdb 連接
%load_ext sql

%sql conn_duckdb

In [ ]:
%%sql
-- 方法1：查詢當前數據庫中的所有表
SHOW TABLES;

## 美股大盤收盤資訊  

## FUNCTION BLOCK


In [9]:
def get_from_histock(target_url):
    """ 從 histock 取得資料  

        target_url: 目標網站
        return: dataframe
    """

    # 設定 User-Agent 來模擬瀏覽器請求
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # 發送請求獲取網頁內容
    response = requests.get(target_url, headers=headers)
    response.encoding = 'utf-8'  # 設定編碼

    # 使用 BeautifulSoup 解析 HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    target_table = soup.find_all('table')[1]

    if target_table is not None:
        table_html = str(target_table)
        # histock_df = pd.read_html(table_html)[0]
        histock_df = pd.read_html(StringIO(table_html))[0]
        
        # 處理漲跌欄位
        def _process_change(value):
            if pd.isna(value):
                return value
            value = str(value)
            if '▲' in value:
                return float(value.replace('▲', ''))
            elif '▼' in value:
                return -float(value.replace('▼', ''))
            return float(value) if value else 0
    
        # 處理比例欄位
        def _process_percentage(value):
            if pd.isna(value):
                return value
            return float(str(value).replace('%', ''))
    
        # 套用轉換函數
        histock_df['漲跌'] = histock_df['漲跌'].apply(_process_change)
        histock_df['比例'] = histock_df['比例'].apply(_process_percentage)
        
        # 移除所有 NaN 的行
        histock_df = histock_df.dropna()
    
    else:
        print("找不到目標表格")

    return histock_df


## MAIN

In [ ]:
# 取得道瓊指數資料
dji_df = get_from_histock('https://histock.tw/index/DJI')

# # 日期欄位轉換為日期格式
# dji_df['日期'] = pd.to_datetime(dji_df['日期'])

# 欄位名稱rename
dji_df.rename(
    columns={
        '日期': 'Date', 
        '指數': '道瓊指數', 
        '漲跌': '道瓊漲跌數', 
        '比例': '道瓊漲跌比例'
    }
    , inplace=True
)

dji_df.head(3)

In [ ]:
# 取得NASDAQ指數資料
nasdaq_df = get_from_histock('https://histock.tw/index/NASDAQ')

# 欄位名稱rename
nasdaq_df.rename(
    columns={
        '日期': 'Date', 
        '指數': '納斯達克指數', 
        '漲跌': '納斯達克漲跌數', 
        '比例': '納斯達克漲跌比例'
    }
    , inplace=True
)

nasdaq_df.head(3)

In [ ]:
# 取得費城半導體資料
sox_df = get_from_histock('https://histock.tw/index/SOX')

# 欄位名稱rename
sox_df.rename(
    columns={
        '日期': 'Date', 
        '指數': '費城半導體指數', 
        '漲跌': '費城半導體漲跌數', 
        '比例': '費城半導體漲跌比例'
    }
    , inplace=True
)

sox_df.head(3)

In [ ]:
# 取得台積電ＡＤＲ資料 
tsmc_adr_df = get_from_histock('https://histock.tw/index/TSM')

# 欄位名稱rename
tsmc_adr_df.rename(
    columns={
        '日期': 'Date', 
        '指數': '台積電ADR指數', 
        '漲跌': '台積電ADR漲跌數', 
        '比例': '台積電ADR漲跌比例'
    }
    , inplace=True
)

tsmc_adr_df.head(3)

In [ ]:
# 取得美元指數資料 
usdx_df = get_from_histock('https://histock.tw/index/USDX')

# 欄位名稱rename
usdx_df.rename(
    columns={
        '日期': 'Date', 
        '指數': '美元指數', 
        '漲跌': '美元指數漲跌數', 
        '比例': '美元指數漲跌比例'
    }
    , inplace=True
)

usdx_df.head(3)


In [ ]:
%%sql result <<
WITH all_dates AS (
    SELECT Date FROM dji_df
    UNION 
    SELECT Date FROM nasdaq_df
    UNION 
    SELECT Date FROM sox_df
    UNION 
    SELECT Date FROM tsmc_adr_df
    UNION 
    SELECT Date FROM usdx_df
)
SELECT 
    d.Date,
    COALESCE(a.道瓊指數, 0) AS 道瓊指數, 
    COALESCE(a.道瓊漲跌數, 0) AS 道瓊漲跌數, 
    COALESCE(a.道瓊漲跌比例, 0) AS 道瓊漲跌比例,
    
    COALESCE(b.納斯達克指數, 0) AS 納斯達克指數, 
    COALESCE(b.納斯達克漲跌數, 0) AS 納斯達克漲跌數, 
    COALESCE(b.納斯達克漲跌比例, 0) AS 納斯達克漲跌比例,

    -- 多空分數～道瓊與那斯達克一起看，兩項同增就多，同減為空
    CASE 
        WHEN a.道瓊漲跌數 > 0 AND b.納斯達克漲跌數 > 0 THEN 1
        WHEN a.道瓊漲跌數 < 0 AND b.納斯達克漲跌數 < 0 THEN -1
        ELSE 0
    END AS 道瓊_納斯達克_多空分數,
    
    COALESCE(c.費城半導體指數, 0) AS 費城半導體指數, 
    COALESCE(c.費城半導體漲跌數, 0) AS 費城半導體漲跌數, 
    COALESCE(c.費城半導體漲跌比例, 0) AS 費城半導體漲跌比例,

    -- 多空分數～費半要看有沒有漲跌超過2%，才做標示，漲超過算多，跌超過算空，其他狀況不做標示
    CASE 
        WHEN c.費城半導體漲跌比例 > 2 THEN 1
        WHEN c.費城半導體漲跌比例 < -2 THEN -1
        ELSE 0
    END AS 費城半導體_多空分數,
    
    COALESCE(e.台積電ADR指數, 0) AS 台積電ADR指數, 
    COALESCE(e.台積電ADR漲跌數, 0) AS 台積電ADR漲跌數, 
    COALESCE(e.台積電ADR漲跌比例, 0) AS 台積電ADR漲跌比例,
    
    -- 多空分數～台積電ＡＤＲ要看有沒有漲跌超過1%，來標示多空，沒超過就都不做標示
    CASE 
        WHEN e.台積電ADR漲跌比例 > 1 THEN 1
        WHEN e.台積電ADR漲跌比例 < -1 THEN -1
        ELSE 0
    END AS 台積電ADR_多空分數,

    COALESCE(f.美元指數, 0) AS 美元指數, 
    COALESCE(f.美元指數漲跌數, 0) AS 美元指數漲跌數, 
    COALESCE(f.美元指數漲跌比例, 0) AS 美元指數漲跌比例
FROM 
    all_dates d
LEFT JOIN 
    dji_df a ON d.Date = a.Date
LEFT JOIN 
    nasdaq_df b ON d.Date = b.Date
LEFT JOIN 
    sox_df c ON d.Date = c.Date
LEFT JOIN 
    tsmc_adr_df e ON d.Date = e.Date
LEFT JOIN 
    usdx_df f ON d.Date = f.Date
ORDER BY 
    d.Date DESC

In [ ]:
# 將 result 轉換為 DataFrame
us_market_df = result.DataFrame()

us_market_df.head(3)


In [ ]:
%%sql
-- 先检查表是否存在，如果不存在则创建
CREATE TABLE IF NOT EXISTS tw_sotck_barometer_part_03 (
    Date VARCHAR,
    道瓊指數 DOUBLE,
    道瓊漲跌數 DOUBLE,
    道瓊漲跌比例 DOUBLE,
    納斯達克指數 DOUBLE,
    納斯達克漲跌數 DOUBLE,
    納斯達克漲跌比例 DOUBLE,
    道瓊_納斯達克_多空分數 INTEGER,
    費城半導體指數 DOUBLE,
    費城半導體漲跌數 DOUBLE,
    費城半導體漲跌比例 DOUBLE,
    費城半導體_多空分數 INTEGER,
    台積電ADR指數 DOUBLE,
    台積電ADR漲跌數 DOUBLE,
    台積電ADR漲跌比例 DOUBLE,
    台積電ADR_多空分數 INTEGER,
    美元指數 DOUBLE,
    美元指數漲跌數 DOUBLE,
    美元指數漲跌比例 DOUBLE,
    PRIMARY KEY (Date)
);

-- 使用UPSERT方式插入数据
INSERT OR REPLACE INTO tw_sotck_barometer_part_03
SELECT * FROM us_market_df;

In [ ]:
%%sql
SELECT * FROM tw_sotck_barometer_part_03 ORDER BY Date DESC LIMIT 10;

In [ ]:
%%sql
SELECT COUNT(*) FROM tw_sotck_barometer_part_03;

In [20]:
# 關閉資料庫連線
conn_duckdb.close()